In [1]:
import ast
import os

def extract_classes(file_path):
    """
    Args:
        file_path: path to a .py file containing ORM classes
    Returns:
        classes: list of tuples containing class names and their methods/attributes.
    """
    with open(file_path, 'r') as file:
        tree = ast.parse(file.read(), filename=file_path)

    classes = []

    for node in tree.body:
        if isinstance(node, ast.ClassDef):
            class_name = node.name
            class_body = []
            inside_special_method = False

            for item in node.body:
                if isinstance(item, ast.FunctionDef) and item.name != '__init__':
                    inside_special_method = True
                if not inside_special_method:
                    class_body.append(item)

            classes.append((class_name, class_body, node.lineno))

    return classes

def print_class(source_lines, class_name, body, start_line):
    """
    Prints the class name and its methods/attributes, preserving original formatting.
    
    Args:
        source_lines: list of lines from the source file.
        class_name: name of the class.
        body: list of nodes representing methods/attributes in the class.
        start_line: line number where the class starts.
    """
    print(f"class {class_name}:")

    for node in body:
        node_lines = ast.get_source_segment("\n".join(source_lines), node).splitlines()
        for line in node_lines:
            # Print the correct indentation
            print(f"    {line}")

        # Print any lines between the current node and the next one
        if node is not body[-1]:
            next_line = body[body.index(node) + 1].lineno - 1
            for line in source_lines[node.end_lineno:next_line]:
                print(f"    {line.strip()}")
    return print() 

def display_orm(directory, tables=[]):
    """
    Args:
        directory: path to the directory with ORM .py files containing the classes.
        tables: list of table (file) names to process.
    Returns:
        prints relevant structural ORM components for easy comparison with SQL.
    """
    for table in tables:
        file_path = os.path.join(directory, table + '.py')

        # Read the file and extract class information
        with open(file_path, 'r') as file:
            source = file.read()
            source_lines = source.splitlines()

        classes_with_bodies = extract_classes(file_path)

        # Print each class
        for class_name, body, start_line in classes_with_bodies:
            print_class(source_lines, class_name, body, start_line)
      
